<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Website-A/B-Testing---Lab" data-toc-modified-id="Website-A/B-Testing---Lab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Website A/B Testing - Lab</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Exploratory-Analysis" data-toc-modified-id="Exploratory-Analysis-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Exploratory Analysis</a></span></li><li><span><a href="#Conduct-a-Statistical-Test" data-toc-modified-id="Conduct-a-Statistical-Test-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Conduct a Statistical Test</a></span></li><li><span><a href="#Verifying-Results" data-toc-modified-id="Verifying-Results-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Verifying Results</a></span></li><li><span><a href="#$n\bullet-p-(1-p)$" data-toc-modified-id="$n\bullet-p-(1-p)$-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>$n\bullet p (1-p)$</a></span><ul class="toc-item"><li><span><a href="#Step-1:" data-toc-modified-id="Step-1:-1.6.1"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>Step 1:</a></span></li><li><span><a href="#Step-2:" data-toc-modified-id="Step-2:-1.6.2"><span class="toc-item-num">1.6.2&nbsp;&nbsp;</span>Step 2:</a></span></li><li><span><a href="#Step-3:" data-toc-modified-id="Step-3:-1.6.3"><span class="toc-item-num">1.6.3&nbsp;&nbsp;</span>Step 3:</a></span></li><li><span><a href="#Analysis:" data-toc-modified-id="Analysis:-1.6.4"><span class="toc-item-num">1.6.4&nbsp;&nbsp;</span>Analysis:</a></span></li></ul></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Summary</a></span></li></ul></li></ul></div>

# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
import csv
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [18]:
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [39]:
clickers = set(df[df['action'] == 'click']['id'].unique())
viewers = set(df[df['action'] == 'view']['id'].unique())
print('This many clicked without viewing: {}'.format(len(clickers-viewers))
print('This many viewed without clicking: {}'.format(len(viewers-clickers)))

This many clicked without viewing: 0
This many viewed without clicking: 4468


In [43]:
control = set(df[df['group'] == 'control']['id'].unique())
exp = set(df[df['group'] == 'experiment']['id'].unique())
print('This many are in the control: {}'.format(len(control)))
print('This many are in the experiment: {}'.format(len(exp)))
print('This many are in both the experiment and control: {}'.format(
    len(exp.intersection(control))))

This many are in the control: 3332
This many are in the experiment: 2996
This many are in both the experiment and control: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [55]:
df['count'] = 1

In [64]:
#Convert clicks into a binary variable on a user-by-user-basis
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

exp = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
exp = exp.fillna(value=0)

print("Sample sizes:\tControl: {}\tExperiment: {}"
      .format(len(control), len(exp)))
print("Total Clicks:\tControl: {}\tExperiment: {}"
      .format(control.click.sum(), exp.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}"
      .format(control.click.mean(), exp.click.mean()))
control.head()

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [65]:
expected_exp_clicks = (control.click.mean()) * (len(exp))
expected_exp_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [75]:
n = len(exp)
p = control.click.mean()
var = n * p * (1-p)
std = np.sqrt(var)
std

24.568547907005815

In [87]:
actual_experiment_clicks = exp.click.sum()
z_score = (actual_experiment_clicks - expected_exp_clicks)/std
print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [88]:
import scipy.stats as stats
p_val = stats.norm.sf(z_score) #or 1 - stats.norm.cdf(z_score)
print(p_val)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **with the p value being less than 0.0025, yes it does confirm
the confidence in the redesign resulting in more clicks. It matches the z
score reading. The z score indicates there is a considerable increase because 
it is more than 3 standard deviations away from the expected (99% of all data
should fall within 3 standard deviations)**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.